# M3. Actividad

Erick Calderón Reyes / A01351361

# - El programa tiene con agentes a los Carros, el semaforo y lo que seria un obstaculo que en este caso es como simulador de banqueta o edificio.

# - El programa tiene como interacción la intersección en un semaforo de una solo linea, en el cual al estar en verde los carros se mueven hacia abajo/arriba y en rojo se mueven los de deracha a izquierda, sin tocar el obstaculo.

In [124]:
import agentpy as ap
import matplotlib.pyplot as plt
import IPython
import random
import itertools
import numpy as np
import json

In [125]:
# Type of move 
LEFT_RIGHT_MOVE = 1
RIGHT_LEFT_MOVE = 2
UP_DOWN_MOVE = 3
DOWN_UP_MOVE = 4

VALID_MOVES = {
    LEFT_RIGHT_MOVE: (0, 1),
    RIGHT_LEFT_MOVE: (0, -1),
    UP_DOWN_MOVE: (1,0),
    DOWN_UP_MOVE: (-1,0)
}

# Colors
CAR_COLOR = 1
YELLOW_COLOR = 101
GREEN_COLOR = 102
RED_COLOR = 103
GRAY_COLOR = 104
BROWN_COLOR = 105

ID = 0

In [126]:
class CarAgent(ap.Agent):
    def setup(self):
        # return value of the type of move
        self.current_move_type = random.choice(list(VALID_MOVES.keys()))
        self.id = 0
        self.x=0
        self.y=0
        self.z=0
        
        self.speed = 0.2
        self.crossed = False

        
    def get_move(self):
        # Allows cars to randomize speed 
        #random.uniform(0, 1)
        #print(self.y)
        if self.speed > 0.1:
            # Returns the coordinate of the choosen move
            return VALID_MOVES[self.current_move_type]
        
        return(0,0)


In [127]:
class LightAgent(ap.Agent):
    def setup(self):

        # Iitilize the default color
        self.current_color = GREEN_COLOR
        
        self.step_time = 0.1         #
        
        # Extra colors
        self.green_time = 10
        self.red_time = 8
        self.yellow_time = 3

    def update(self):
        self.state_time += self.step_time

        if self.state == 0:
            # GREEN STATE
            if self.state_time >= self.green_time:
                self.state = 1
                self.current_color = YELLOW_COLOR
                self.state_time = 0
        elif self.state == 1:
            # YELLOW STATE
            if self.state_time >= self.yellow_time:
                self.state = 2
                self.current_color = RED_COLOR
                self.state_time = 0
        elif self.state == 2:
            # RED STATE
            if self.state_time >= self.red_time:
                self.state = 0
                self.current_color = GREEN_COLOR
                self.state_time = 0

    def set_green(self):     
        self.state = 0
        self.current_color = GREEN_COLOR
        self.state_time = 0

    def set_yellow(self):     
        self.state = 1
        self.current_color = YELLOW_COLOR
        self.state_time = 0

    def set_red(self):     
        self.state = 2
        self.current_color = RED_COLOR
        self.state_time = 0

In [128]:
class SidewalkAgent(ap.Agent):
    def setup(self):
        self.current_color = GRAY_COLOR

In [129]:
location_lights = itertools.product(range(5,50,5), range(10,100,10))
len(list(location_lights))

81

In [130]:
class StreetModel(ap.Model):
    def setup(self):
        self.x_car =[]
        self.y_car =[]
        self.matrix =[]
        global initial_car_location
        #Create  agents
        
        #Create first the X lights
        x_separation = self.p.lights_separation["X"]
        y_separation = self.p.lights_separation["Y"]

        x_valid = range(x_separation, self.p.size["X"], x_separation)
        y_valid = range(y_separation, self.p.size["Y"], y_separation)

        lights_location = list(itertools.product(y_valid, x_valid))
        n_lights = len(lights_location)

        self.lights = ap.AgentList(self, 1, LightAgent)
        self.lights.is_car = False
        self.lights.type_agent = YELLOW_COLOR
        
        self.lights.step_time =  self.p.step_time
        self.lights.green_time = self.p.green
        self.lights.yellow_time = self.p.yellow
        self.lights.red_time = self.p.red
        

        self.cars = ap.AgentList(self, int(self.p["cars"]), CarAgent)
        iterator = 0

        self.current_car_positions = []

        for car in self.cars:
            car.id = iterator
            iterator += 1
            carObject = {
                    'id': car.id,
                    'x':0,
                    'y':0,
                }
            self.current_car_positions.append(carObject)
        self.cars.is_car = True
        self.cars.type_agent = CAR_COLOR

        #create grid (street)
        self.street = ap.Grid(self, [self.p["size"]["Y"], self.p["size"]["X"]], track_empty=True, torus=True)
        
        n_sidewalks = 5
        self.sidewalk = ap.AgentList(self, n_sidewalks, SidewalkAgent)
        self.sidewalk.is_car = False
        self.sidewalk.type_agent = BROWN_COLOR
        
        # add sidewalks
        for a in range(3):
            for b in range(4):
                s_list = []
                sidewalk_location_1 = a, b
                s_list.append(sidewalk_location_1)
                if s_list[0] == lights_location[0]:
                    s_list.pop(0)
                self.street.add_agents(self.sidewalk, s_list)
        
                
        #add the lights agents
        self.street.add_agents(self.lights, lights_location)
        

        valid_car_location = [(5, 0), (0, 5)]
        
        initial_car_location = [random.choice(valid_car_location) for _ in self.cars]
        

        for car, initial_location in zip(self.cars, initial_car_location):
            
            if initial_location[0] == 5:
                #up
                car.current_move_type = LEFT_RIGHT_MOVE
            elif initial_location[0] == 0:
                #right
                car.current_move_type = UP_DOWN_MOVE
        
        self.street.add_agents(self.cars, initial_car_location)


        
    def step(self):
        iterator = 0
        global initial_car_location
        for car in self.cars:
            coordinates_move = car.get_move()
            
            if(coordinates_move==(0, 1)):
                car.x = car.x+1
                self.x_car += [car.x]
            elif(coordinates_move==(0, -1)):
                car.x = car.x-1
                self.x_car += [car.x]
            if(coordinates_move==(1,0)):
                car.y = car.y+1
                self.y_car += [car.y]
            elif(coordinates_move==(-1, 0)):
                car.y = car.y-1
                self.y_car += [car.y]
                
            
            self.current_car_positions[iterator]['x'] = car.x
            self.current_car_positions[iterator]['y'] = car.y

            iterator +=1
        json_string = json.dumps(self.current_car_positions)
        
        if self.p.step_time <= self.p.green:
            self.lights[0].type_agent = GREEN_COLOR

            for ctr in range(len(initial_car_location)):
                if initial_car_location[ctr][0] == 0:
                    coordinates_move = self.cars[ctr].get_move()
                    self.street.move_by(self.cars[ctr], coordinates_move)
                elif initial_car_location[ctr][0] == 5:
                    coordinates_move = (0,0)
                    self.street.move_by(self.cars[ctr], coordinates_move)
                else:
                    pass
            
        elif self.p.step_time > self.p.green and self.p.step_time <= self.p.yellow:
            speed = 1
            self.lights[0].type_agent = YELLOW_COLOR


            for ctr in range(len(initial_car_location)):
                if initial_car_location[ctr][0] == 0:
                    coordinates_move = self.cars[ctr].get_move()
                    self.street.move_by(self.cars[ctr], (speed*coordinates_move[0], speed*coordinates_move[1]))
                elif initial_car_location[ctr][0] == 5:
                    coordinates_move = (0,0)
                    self.street.move_by(self.cars[ctr], coordinates_move)
                else:
                    pass
            
        else:
            
            self.lights[0].type_agent = RED_COLOR


            for ctr in range(len(initial_car_location)):
                if initial_car_location[ctr][0] == 0:
                    coordinates_move = (0,0)
                    self.street.move_by(self.cars[ctr], coordinates_move)
                elif initial_car_location[ctr][0] == 5:
                    coordinates_move = self.cars[ctr].get_move()
                    self.street.move_by(self.cars[ctr], coordinates_move)
                else:
                    pass
            
        
        self.p.step_time += 1

        
    def end(self):
        pass

In [131]:
# Define parameters

parameters = {
    'step_time': 0.1,
    'cars': 20,
    'lights_separation': {'X': 3, 'Y': 3},
    'size': {'X': 8, 'Y': 8},
    'green': 80.0,          
    'yellow': 120.0,         
    'red': 200.0,
    'steps': 300,
}

In [132]:

def animation_plot(model, ax):
    attr_grid = model.street.attr_grid('type_agent')
    color_dict = {CAR_COLOR:'#0000FF', YELLOW_COLOR:'#FFFF00', GREEN_COLOR:'#00FF00', RED_COLOR:'#FF0000',BROWN_COLOR:'#FFF8DC', GRAY_COLOR: '#A49E9E',None:'#A49E9E'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of a two-way intersection Traffic Lights\n"
                 f"Time-step: {model.t}")

fig, ax = plt.subplots()
model = StreetModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))

In [133]:
model.reporters

{'seed': 308838585530506357253688914871753301865}